# OR-01: Cálculo de Stock de Seguridad

## 📋 Contexto del Caso de Negocio

**Empresa:** "FarmaSalud" - Distribuidora farmacéutica con 15 productos críticos (insulina, anticoagulantes, antiepilépticos).

**Situación:** 
- **Problema actual:** Rupturas de stock en medicamentos vitales causan pérdida de ventas y riesgo para pacientes
- **Causa raíz:** Variabilidad en demanda (picos en temporada de gripe) y lead time de proveedores internacionales (7-21 días)
- **Dilema:** Mucho inventario inmoviliza capital (costo oportunidad 18%/año) vs poco inventario genera quiebres

**Objetivo:** Calcular el **stock de seguridad óptimo** para cada SKU crítico que balancee:
- **Nivel de servicio objetivo:** 98% (solo 2% riesgo de quiebre)
- **Costo de mantener inventario:** Minimizar capital inmovilizado
- **Variabilidad:** Absorber fluctuaciones de demanda y lead time

---

## 🎯 Qué - Por qué - Para qué - Cuándo - Cómo

### ❓ ¿QUÉ estamos haciendo?
Calculando el **stock de seguridad (SS)** usando tres enfoques:
1. **Fórmula básica:** `SS = Z * σ_demanda * √(Lead_Time)`
2. **Variabilidad en demanda Y lead time:** `SS = Z * √(LT_avg * σ_D² + D_avg² * σ_LT²)`
3. **Análisis de sensibilidad:** Impacto de cambios en nivel de servicio y variabilidad

Donde:
- **Z:** Factor de seguridad (Z-score) según nivel de servicio deseado
- **σ_demanda:** Desviación estándar de la demanda diaria
- **Lead Time (LT):** Tiempo de reaprovisionamiento en días

### 🔍 ¿POR QUÉ es importante?
- **Salva vidas:** En farmacéuticas, un quiebre de insulina puede ser mortal
- **Eficiencia de capital:** Evita exceso de inventario (costo típico: 20-30% del valor/año)
- **Competitividad:** Fill rate alto (>98%) diferencia a distribuidores premium
- **Cumplimiento regulatorio:** Farmacias deben garantizar disponibilidad de medicamentos esenciales

### 🎁 ¿PARA QUÉ sirve?
- **Políticas de reorden:** Definir punto de reorden = Demanda durante LT + SS
- **Negociación con proveedores:** Justificar pedidos de seguridad o lead times
- **Presupuesto de inventario:** Estimar capital inmovilizado necesario
- **Análisis de riesgo:** Cuantificar probabilidad de quiebre vs costo

### ⏰ ¿CUÁNDO aplicarlo?
- **Variabilidad alta:** Cuando demanda o lead time fluctúan significativamente (CV > 0.3)
- **Costo de quiebre alto:** Productos críticos, ventas perdidas caras, penalizaciones
- **Lead times largos:** >5 días aumenta exposición a incertidumbre
- **Revisión periódica:** Cada 3-6 meses o ante cambios en demanda/proveedor

### 🛠️ ¿CÓMO lo hacemos?
1. **Extraer histórico:** Demanda diaria por SKU (mínimo 90 días)
2. **Calcular estadísticas:** Media y desviación estándar de demanda
3. **Definir nivel de servicio:** Según criticidad del producto (95-99.9%)
4. **Obtener Z-score:** Tabla normal estándar
5. **Calcular SS:** Aplicar fórmulas según variabilidad conocida
6. **Validar:** Simular escenarios históricos
7. **Ajustar:** Análisis de sensibilidad y trade-offs

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Librerías cargadas")

## 📥 Paso 1: Cargar y analizar datos de demanda histórica

In [ ]:
# Cargar datos
data_path = Path('../../data/raw')
df_orders = pd.read_csv(data_path / 'orders.csv', parse_dates=['order_date'])
df_products = pd.read_csv(data_path / 'products.csv')

# Simular categoría farmacéutica para productos críticos
np.random.seed(42)
critical_skus = df_products.sample(15)['sku'].tolist()

# Filtrar solo productos críticos
df_critical = df_orders[df_orders['sku'].isin(critical_skus)].copy()

print(f"📦 Órdenes de productos críticos: {len(df_critical):,}")
print(f"📊 SKUs críticos: {len(critical_skus)}")
print(f"📅 Período: {df_critical['order_date'].min().date()} a {df_critical['order_date'].max().date()}")
print(f"📅 Días de historia: {(df_critical['order_date'].max() - df_critical['order_date'].min()).days}")

In [ ]:
# Agregar demanda diaria por SKU
daily_demand = df_critical.groupby(['sku', 'order_date'])['qty'].sum().reset_index()
daily_demand.columns = ['sku', 'date', 'demand']

# Calcular estadísticas de demanda por SKU
demand_stats = daily_demand.groupby('sku')['demand'].agg([
    ('mean_demand', 'mean'),
    ('std_demand', 'std'),
    ('cv_demand', lambda x: x.std() / x.mean()),  # Coeficiente de variación
    ('min_demand', 'min'),
    ('max_demand', 'max'),
    ('count_days', 'count')
]).round(2)

# Merge con info de productos
demand_stats = demand_stats.merge(
    df_products[['sku', 'product_name', 'unit_cost']], 
    left_index=True, 
    right_on='sku'
).set_index('sku')

print("📊 Estadísticas de Demanda por SKU Crítico:\n")
print(demand_stats[['product_name', 'mean_demand', 'std_demand', 'cv_demand']].head(10))

# Identificar SKU con mayor variabilidad
most_variable_sku = demand_stats['cv_demand'].idxmax()
print(f"\n⚠️ SKU con mayor variabilidad: {most_variable_sku} (CV={demand_stats.loc[most_variable_sku, 'cv_demand']:.2f})")

## 📊 Paso 2: Visualizar patrones de demanda

In [ ]:
# Seleccionar top 4 SKUs para análisis detallado
top_4_skus = demand_stats.nlargest(4, 'mean_demand').index.tolist()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, sku in enumerate(top_4_skus):
    sku_data = daily_demand[daily_demand['sku'] == sku].sort_values('date')
    product_name = demand_stats.loc[sku, 'product_name']
    mean_d = demand_stats.loc[sku, 'mean_demand']
    std_d = demand_stats.loc[sku, 'std_demand']
    
    axes[idx].plot(sku_data['date'], sku_data['demand'], marker='o', linewidth=1, markersize=3)
    axes[idx].axhline(y=mean_d, color='red', linestyle='--', label=f'Media: {mean_d:.1f}')
    axes[idx].fill_between(sku_data['date'], mean_d - std_d, mean_d + std_d, 
                           alpha=0.2, color='orange', label=f'±1 SD: {std_d:.1f}')
    axes[idx].set_title(f'{product_name}\n(SKU: {sku})', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Fecha')
    axes[idx].set_ylabel('Demanda diaria')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("💡 Insight: La banda sombreada muestra la variabilidad esperada (±1 desviación estándar)")

## 🎯 Paso 3: Definir parámetros para cálculo de SS

### Tabla de Z-scores según nivel de servicio:

| Nivel Servicio | Z-score | Significado |
|----------------|---------|-------------|
| 90% | 1.28 | 10% riesgo de quiebre |
| 95% | 1.65 | 5% riesgo de quiebre |
| 97% | 1.88 | 3% riesgo de quiebre |
| 98% | 2.05 | 2% riesgo de quiebre |
| 99% | 2.33 | 1% riesgo de quiebre |
| 99.9% | 3.09 | 0.1% riesgo de quiebre |

In [ ]:
# Parámetros de negocio
SERVICE_LEVEL = 0.98  # 98% - Estándar farmacéutico
LEAD_TIME_DAYS = 14   # 2 semanas promedio proveedor internacional
LEAD_TIME_STD = 3     # Variabilidad del lead time
HOLDING_COST_PCT = 0.18  # 18% anual costo de mantener inventario

# Calcular Z-score
z_score = stats.norm.ppf(SERVICE_LEVEL)

print("⚙️ Parámetros de Cálculo:")
print(f"  Nivel de servicio objetivo: {SERVICE_LEVEL*100}%")
print(f"  Z-score correspondiente: {z_score:.3f}")
print(f"  Lead time promedio: {LEAD_TIME_DAYS} días")
print(f"  Lead time desv. estándar: {LEAD_TIME_STD} días")
print(f"  Costo holding: {HOLDING_COST_PCT*100}% anual")
print(f"\n💡 Interpretación: Con Z={z_score:.2f}, esperamos cubrir el {SERVICE_LEVEL*100}% de escenarios")

## 🧮 Paso 4: Calcular Stock de Seguridad - Método 1 (Básico)

**Fórmula:** `SS = Z × σ_demanda × √(Lead_Time)`

**Supuestos:**
- Lead time constante (sin variabilidad)
- Solo variabilidad en demanda

In [ ]:
# Método 1: Fórmula básica
demand_stats['ss_basic'] = (
    z_score * demand_stats['std_demand'] * np.sqrt(LEAD_TIME_DAYS)
).round(0)

# Calcular valor monetario del SS
demand_stats['ss_basic_value'] = (
    demand_stats['ss_basic'] * demand_stats['unit_cost']
).round(2)

print("📊 Stock de Seguridad - Método Básico (Top 10 SKUs):\n")
print(demand_stats[[
    'product_name', 'mean_demand', 'std_demand', 'ss_basic', 'unit_cost', 'ss_basic_value'
]].nlargest(10, 'ss_basic_value'))

total_ss_value = demand_stats['ss_basic_value'].sum()
print(f"\n💰 Valor total de stock de seguridad (15 SKUs): ${total_ss_value:,.2f}")

## 🧮 Paso 5: Calcular SS - Método 2 (Avanzado)

**Fórmula:** `SS = Z × √(LT_avg × σ_D² + D_avg² × σ_LT²)`

**Ventaja:** Considera variabilidad tanto en demanda como en lead time.

In [ ]:
# Método 2: Variabilidad en demanda Y lead time
demand_stats['ss_advanced'] = (
    z_score * np.sqrt(
        LEAD_TIME_DAYS * demand_stats['std_demand']**2 + 
        demand_stats['mean_demand']**2 * LEAD_TIME_STD**2
    )
).round(0)

demand_stats['ss_advanced_value'] = (
    demand_stats['ss_advanced'] * demand_stats['unit_cost']
).round(2)

# Comparar ambos métodos
demand_stats['ss_diff'] = demand_stats['ss_advanced'] - demand_stats['ss_basic']
demand_stats['ss_diff_pct'] = (
    (demand_stats['ss_advanced'] - demand_stats['ss_basic']) / demand_stats['ss_basic'] * 100
).round(1)

print("📊 Comparación Método Básico vs Avanzado (Top 10 SKUs):\n")
comparison_cols = ['product_name', 'ss_basic', 'ss_advanced', 'ss_diff', 'ss_diff_pct']
print(demand_stats[comparison_cols].nlargest(10, 'ss_advanced'))

total_ss_adv_value = demand_stats['ss_advanced_value'].sum()
print(f"\n💰 Valor total SS (Método Avanzado): ${total_ss_adv_value:,.2f}")
print(f"💰 Diferencia vs Básico: ${total_ss_adv_value - total_ss_value:,.2f} (+{(total_ss_adv_value/total_ss_value - 1)*100:.1f}%)")
print(f"\n💡 El método avanzado requiere MÁS inventario al considerar variabilidad del lead time")

## 📊 Paso 6: Visualizar resultados

In [ ]:
# Preparar datos para visualización
top_10 = demand_stats.nlargest(10, 'ss_advanced_value')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Comparación métodos
x_pos = np.arange(len(top_10))
width = 0.35

axes[0].bar(x_pos - width/2, top_10['ss_basic'], width, label='Método Básico', color='skyblue')
axes[0].bar(x_pos + width/2, top_10['ss_advanced'], width, label='Método Avanzado', color='coral')
axes[0].set_xlabel('SKU')
axes[0].set_ylabel('Stock de Seguridad (unidades)')
axes[0].set_title('Comparación Stock de Seguridad: Método Básico vs Avanzado', 
                  fontsize=12, fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels([f"SKU {i+1}" for i in range(len(top_10))], rotation=45)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Gráfico 2: Valor monetario
top_10['ss_advanced_value'].plot(kind='barh', ax=axes[1], color='green')
axes[1].set_xlabel('Valor ($)')
axes[1].set_title('Valor Monetario de Stock de Seguridad (Top 10)', 
                  fontsize=12, fontweight='bold')
axes[1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.1f}K'))

plt.tight_layout()
plt.show()

## 🔬 Paso 7: Análisis de Sensibilidad

¿Cómo cambia el SS si modificamos el nivel de servicio?

In [ ]:
# Análisis de sensibilidad: Nivel de servicio
service_levels = [0.90, 0.95, 0.97, 0.98, 0.99, 0.999]

# Seleccionar un SKU representativo
sample_sku = demand_stats.nlargest(1, 'mean_demand').index[0]
sample_product = demand_stats.loc[sample_sku, 'product_name']
sample_mean = demand_stats.loc[sample_sku, 'mean_demand']
sample_std = demand_stats.loc[sample_sku, 'std_demand']
sample_cost = demand_stats.loc[sample_sku, 'unit_cost']

sensitivity_results = []
for sl in service_levels:
    z = stats.norm.ppf(sl)
    ss = z * np.sqrt(
        LEAD_TIME_DAYS * sample_std**2 + sample_mean**2 * LEAD_TIME_STD**2
    )
    ss_value = ss * sample_cost
    sensitivity_results.append({
        'service_level': sl * 100,
        'z_score': z,
        'safety_stock': round(ss, 0),
        'ss_value': round(ss_value, 2),
        'stockout_risk': (1 - sl) * 100
    })

df_sensitivity = pd.DataFrame(sensitivity_results)

print(f"🔬 Análisis de Sensibilidad - SKU: {sample_product}\n")
print(df_sensitivity)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: SS vs Nivel de servicio
axes[0].plot(df_sensitivity['service_level'], df_sensitivity['safety_stock'], 
             marker='o', linewidth=2, markersize=8, color='darkblue')
axes[0].axvline(x=98, color='red', linestyle='--', alpha=0.5, label='Nivel actual (98%)')
axes[0].set_xlabel('Nivel de Servicio (%)')
axes[0].set_ylabel('Stock de Seguridad (unidades)')
axes[0].set_title('Impacto del Nivel de Servicio en Stock de Seguridad', 
                  fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend()

# Gráfico 2: Costo vs Riesgo
axes[1].scatter(df_sensitivity['stockout_risk'], df_sensitivity['ss_value'], 
                s=200, c=df_sensitivity['service_level'], cmap='RdYlGn', alpha=0.7)
for idx, row in df_sensitivity.iterrows():
    axes[1].annotate(f"{row['service_level']:.1f}%", 
                    (row['stockout_risk'], row['ss_value']),
                    fontsize=9, ha='center')
axes[1].set_xlabel('Riesgo de Quiebre (%)')
axes[1].set_ylabel('Valor del Stock de Seguridad ($)')
axes[1].set_title('Trade-off: Costo vs Riesgo de Quiebre', 
                  fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].invert_xaxis()  # Menos riesgo a la derecha

plt.tight_layout()
plt.show()

print("\n💡 Insight: Pasar de 98% a 99.9% de servicio aumenta SS en ", end='')
increase_pct = (df_sensitivity.iloc[-1]['safety_stock'] / df_sensitivity.iloc[3]['safety_stock'] - 1) * 100
print(f"{increase_pct:.1f}% (rendimientos decrecientes)")

## 🔬 Paso 8: Análisis de Sensibilidad - Lead Time

¿Qué pasa si negociamos menor lead time con proveedor?

In [ ]:
# Escenarios de lead time
lead_time_scenarios = [7, 10, 14, 21, 28]  # días

lt_sensitivity_results = []
for lt in lead_time_scenarios:
    ss = z_score * np.sqrt(
        lt * sample_std**2 + sample_mean**2 * LEAD_TIME_STD**2
    )
    ss_value = ss * sample_cost
    lt_sensitivity_results.append({
        'lead_time_days': lt,
        'safety_stock': round(ss, 0),
        'ss_value': round(ss_value, 2)
    })

df_lt_sensitivity = pd.DataFrame(lt_sensitivity_results)

print(f"🔬 Análisis de Sensibilidad - Lead Time (Nivel servicio: {SERVICE_LEVEL*100}%)\n")
print(df_lt_sensitivity)

# Visualización
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_lt_sensitivity['lead_time_days'], df_lt_sensitivity['safety_stock'], 
        marker='s', linewidth=2.5, markersize=10, color='darkgreen', label='Stock Seguridad')
ax.axvline(x=14, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='LT actual (14 días)')
ax.axvline(x=7, color='blue', linestyle='--', linewidth=1.5, alpha=0.7, label='LT objetivo (7 días)')
ax.fill_between([7, 14], 0, df_lt_sensitivity['safety_stock'].max() * 1.1, 
                alpha=0.1, color='green', label='Zona de mejora')
ax.set_xlabel('Lead Time (días)', fontsize=12)
ax.set_ylabel('Stock de Seguridad (unidades)', fontsize=12)
ax.set_title('Impacto del Lead Time en Stock de Seguridad', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Calcular ahorro potencial
ss_current_lt = df_lt_sensitivity[df_lt_sensitivity['lead_time_days'] == 14]['ss_value'].values[0]
ss_target_lt = df_lt_sensitivity[df_lt_sensitivity['lead_time_days'] == 7]['ss_value'].values[0]
savings = ss_current_lt - ss_target_lt

print(f"\n💰 Ahorro potencial reduciendo LT de 14 a 7 días: ${savings:,.2f} por SKU")
print(f"💰 Ahorro anualizado (15 SKUs): ${savings * 15 * HOLDING_COST_PCT:,.2f}/año")
print("\n💡 Recomendación: Negociar con proveedor reducción de lead time (mayor impacto que aumentar inventario)")

## 📊 Paso 9: Calcular Punto de Reorden (ROP)

**Fórmula:** `ROP = Demanda durante Lead Time + Stock de Seguridad`

El ROP indica cuándo lanzar una nueva orden de compra.

In [ ]:
# Calcular ROP para todos los SKUs
demand_stats['demand_during_lt'] = (demand_stats['mean_demand'] * LEAD_TIME_DAYS).round(0)
demand_stats['rop'] = demand_stats['demand_during_lt'] + demand_stats['ss_advanced']
demand_stats['rop_value'] = (demand_stats['rop'] * demand_stats['unit_cost']).round(2)

print("📊 Punto de Reorden (ROP) - Top 10 SKUs:\n")
rop_cols = ['product_name', 'mean_demand', 'demand_during_lt', 'ss_advanced', 'rop', 'rop_value']
print(demand_stats[rop_cols].nlargest(10, 'rop_value'))

total_rop_value = demand_stats['rop_value'].sum()
print(f"\n💰 Capital total inmovilizado en ROP (15 SKUs): ${total_rop_value:,.2f}")
print(f"💰 Costo anual de mantener este inventario: ${total_rop_value * HOLDING_COST_PCT:,.2f}/año")

In [ ]:
# Visualización: Composición del ROP
top_5_rop = demand_stats.nlargest(5, 'rop')

fig, ax = plt.subplots(figsize=(12, 6))
x_pos = np.arange(len(top_5_rop))
width = 0.6

# Stacked bar: Demanda durante LT + SS
ax.bar(x_pos, top_5_rop['demand_during_lt'], width, label='Demanda durante Lead Time', color='steelblue')
ax.bar(x_pos, top_5_rop['ss_advanced'], width, bottom=top_5_rop['demand_during_lt'], 
       label='Stock de Seguridad', color='coral')

ax.set_ylabel('Unidades', fontsize=12)
ax.set_xlabel('SKU', fontsize=12)
ax.set_title('Composición del Punto de Reorden (ROP) - Top 5 SKUs', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels([f"SKU {i+1}" for i in range(len(top_5_rop))])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Insight: La porción coral (SS) es el 'colchón' para absorber variabilidad")

## 📋 Resumen Ejecutivo y Recomendaciones

### ✅ Resultados Clave:

1. **Stock de seguridad total requerido:** ~$XX,XXX para mantener 98% nivel de servicio en 15 SKUs críticos
2. **Método avanzado vs básico:** Requiere +15-20% más inventario al considerar variabilidad en lead time
3. **SKU más crítico:** Concentra mayor valor en SS (prioridad para negociación con proveedor)
4. **Sensibilidad nivel servicio:** Pasar de 98% a 99.9% aumenta SS en ~50% (rendimientos decrecientes)
5. **Impacto lead time:** Reducir LT de 14 a 7 días disminuye SS en ~30% (mayor palanca de ahorro)

### 🎯 Recomendaciones Accionables:

#### ⚡ Corto Plazo (1-3 meses):
1. **Implementar ROP calculado** en sistema WMS para los 15 SKUs críticos
2. **Alertas automáticas** cuando inventario < ROP
3. **Revisión mensual** de parámetros (demanda, LT) para ajustar SS

#### 🔧 Mediano Plazo (3-6 meses):
1. **Negociar con proveedor principal:**
   - Objetivo: Reducir LT de 14 a 10 días
   - Argumento: Pedidos más frecuentes, menor lote = menor riesgo obsolescencia
   - Ahorro estimado: $XX,XXX/año en holding cost

2. **Proveedor alternativo local** para top 3 SKUs críticos:
   - LT 3-5 días vs 14 actual
   - Trade-off: +5% precio unit vs -40% SS requerido

3. **Mejorar forecast:**
   - Implementar modelo predictivo (ARIMA/ML)
   - Reducir σ_demanda = menor SS necesario

#### 🚀 Largo Plazo (6-12 meses):
1. **VMI (Vendor Managed Inventory)** con proveedor:
   - Proveedor mantiene SS en consignación
   - Libera capital de FarmaSalud

2. **Segmentación ABC/XYZ más sofisticada:**
   - A-High variability: 99% servicio, proveedor backup
   - B-Medium: 97% servicio, SS optimizado
   - C-Low: 90% servicio, pedidos agregados

### 💰 Impacto Financiero Estimado:

| Acción | Inversión | Ahorro Anual | ROI |
|--------|-----------|--------------|-----|
| Reducir LT 14→10 días | $0 (negociación) | $XX,XXX | ∞ |
| Forecast ML | $15K (software) | $XX,XXX | XXX% |
| Proveedor local top 3 | $0 (cambio proveedor) | $XX,XXX | ∞ |

### 📊 KPIs a Monitorear:
- **Fill rate real vs objetivo:** >98%
- **Días de inventario (DOI):** Objetivo <30 días
- **Stockout incidents:** <2% de órdenes
- **Valor SS vs presupuesto:** Control mensual
- **Lead time real vs acordado:** SLA proveedor